# 8.4. Análisis de rendimiento y Vectorización.

In [ ]:
import numpy as np

Ejemplos de cómo vectorizar progresivamente una instrucción:

```R
rent_activos[2,1]<-log(activos[2,1]/activos[1,1]) # 1 activo un día
rent_activos[2,1:length(activos)] <- log(activos[2,1:length(activos)]/activos[2-1,1:length(activos)]) # Todos los act un solo día (día 2)
rent_activos[2:5,1:length(activos)] <- log(activos[2:5,1:length(activos)]/activos[(2:5)-1,1:length(activos)]) # Todos los act días 2 a 5
 
# Todos los activos todos los días
rent_activos[2:dim(activos)[1],1:length(activos)] <-log(activos[2:dim(activos)[1],1:length(activos)]/activos[2:dim(activos)[1]-1,1:length(activos)])
```


## Con funciones de numpy

- Para hacer eso existe la funcion numpy.diff.
- La indexación directa la intentamos evitar lo máximo posible por que solo la persona que ha escrito ese código sabe lo que hace y al resto tenemos que perder el tiempo para enterderlo
- Solo lo usamos cuando no queda más remedio (no tenemos funciones que lo hagan)

In [ ]:
# para un único actiovo
p_activo = np.linspace(1,20,50)
print(p_activo)

r_activo = np.diff(np.log(p_activo))
print(r_activo)

In [ ]:
# para un varios activos
matrix_price = np.random.uniform(0,20, (50,5))
print(matrix_price)

# se aplica diff por filas
matrix_return = np.diff(np.log(matrix_price), axis=0)
matrix_return

## Con Index
Como lo pone Guillermo se hace exactamente de la misma manera

In [ ]:
matrix_price.shape # dias x activos

``` R
rent_activos[2,1]<-log(activos[2,1]/activos[1,1]) # 1 activo un día
```

In [ ]:
np.log(matrix_price[1,0]/matrix_price[0,0])

In [ ]:
# mismo resultado
matrix_return[0,0]

en caso de querer guardarlos se hace de la misma manera pero definiendo la matriz ret_activos como zeros: 

In [ ]:
rent_activos = np.zeros((matrix_price.shape[0]-1, matrix_price.shape[1]))

In [ ]:
rent_activos[0, 0] = np.log(matrix_price[1,0]/matrix_price[0,0])

``` R
rent_activos[2,1:length(activos)] <- log(activos[2,1:length(activos)]/activos[2-1,1:length(activos)]) # Todos los act un solo día (día 2)
```

In [ ]:
np.log(matrix_price[1,:] / matrix_price[0,:]) # no hace fata el length por que tenemos los dos puntos :

``` R
rent_activos[2:5,1:length(activos)] <- log(activos[2:5,1:length(activos)]/activos[(2:5)-1,1:length(activos)]) # Todos los act días 2 a 5
```

In [ ]:
np.log(matrix_price[1:6,:] /matrix_price[0:5,:])

In [ ]:
# mismo resultado
matrix_return[:5,:]

``` R
# Todos los activos todos los días
rent_activos[2:dim(activos)[1],1:length(activos)] <-log(activos[2:dim(activos)[1],1:length(activos)]/activos[2:dim(activos)[1]-1,1:length(activos)])
```


In [ ]:
result = np.log(matrix_price[1:,:] / matrix_price[0:-1,:])

In [ ]:
result[:5, :]

In [ ]:
result[-5:, :]

In [ ]:
# mismo resultado
matrix_return[-5:,:]

#### En pandas

In [ ]:
# en pandas se hace de la misma manera
import pandas as pd
serie = pd.Series(np.linspace(1,20,50))

In [ ]:
retornos = np.log(serie).diff()
retornos

In [ ]:
df = pd.DataFrame({f"asset: {i}": np.linspace(1,20,50) for i in range(5)})

In [ ]:
retornos = np.log(df).diff()
retornos

## 50.000 combinaciones de lotería. Objetivo: Análisis de rendimiento del programa.

___

In [ ]:
combinaciones = 50000
n_bolas = 5

### Solo usando modulos del lenguaje 

In [ ]:
import random

In [ ]:
def sacar_bola_slow(combi, n_bola):
    "esta forma pued creo que puede producir bolas repetidas"
    bola = random.randint(0, 50)
    for bola_comp in range(n_bola):
        if bola == combi[bola_comp]:
            bola = random.randint(0, 50)
    return bola

In [ ]:
%%time
combi_ganadora = []
for i in range(n_bolas):
    combi_ganadora.append(sacar_bola_slow(combi_ganadora, i))

apuestas = []
aciertos = []

for combinancion in range(combinaciones):
    combi_apostada = []
    for n_bola in range(n_bolas):
        combi_apostada.append(sacar_bola_slow(combi_apostada, n_bola))
    apuestas.append(combi_apostada)
    
    aciertos_combinacion = 0
    for bola_ganadora in combi_ganadora:
        for bola_apostada in combi_apostada:
            if bola_ganadora == bola_apostada:
                aciertos_combinacion += 1
    aciertos.append(aciertos_combinacion)

for num_aciertos in range(n_bolas):
    print(f"{num_aciertos} : {aciertos.count(num_aciertos)}")

### Podemos mejorar la funcion sacar bola con recursion

In [ ]:
def sacar_bola(combi):
    "usa recursion, mo produce repetias"
    bola = random.randint(0, 50)
    if bola in combi:
        bola = sacar_bola(combi)
    return bola

In [ ]:
%%time
combi_ganadora = []
for i in range(n_bolas):
    combi_ganadora.append(sacar_bola(combi_ganadora))

apuestas = []
aciertos = []

for combinancion in range(combinaciones):
    combi_apostada = []
    for n_bola in range(n_bolas):
        combi_apostada.append(sacar_bola(combi_apostada))
    apuestas.append(combi_apostada)
    
    aciertos_combinacion = 0
    for bola_ganadora in combi_ganadora:
        for bola_apostada in combi_apostada:
            if bola_ganadora == bola_apostada:
                aciertos_combinacion += 1
    aciertos.append(aciertos_combinacion)

for num_aciertos in range(n_bolas):
    print(f"{num_aciertos} : {aciertos.count(num_aciertos)}")

#### Podemos sustituir por sets los dos bucles del final

In [ ]:
%%time
combi_ganadora = []
for i in range(5):
    combi_ganadora.append(sacar_bola(combi_ganadora))

apuestas = []
aciertos = []

for combinancion in range(combinaciones):
    combi_apostada = []
    for n_bola in range(5):
        combi_apostada.append(sacar_bola(combi_apostada))
    apuestas.append(combi_apostada)
    
    aciertos_combinacion = len(set(combi_apostada).intersection(combi_ganadora))
    aciertos.append(aciertos_combinacion)

for num_aciertos in range(5):
    print(f"{num_aciertos} : {aciertos.count(num_aciertos)}")

### Con numpy

In [ ]:
import numpy as np

In [ ]:
%%time
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

aciertos = np.zeros(combinaciones)

for i in range(combinaciones):
    combinacion = np.random.choice(50, 5, replace=True)
    combi_apostada = np.zeros(50)
    combi_apostada[combinacion] = 1
    
    aciertos[i] = combi_ganadora@combi_apostada

unique, counts = np.unique(aciertos, return_counts=True)
print(f"{unique} {counts}")

Podemos definir las apuestas antes:

In [ ]:
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

apuestas = np.zeros((combinaciones, 50))
apuestas[:, :5] = 1
[np.random.shuffle(apuesta) for apuesta in apuestas]

num_aciertos = apuestas @ combi_ganadora
unique, counts = np.unique(num_aciertos, return_counts=True)
print(f"{unique} {counts}")

aun así sigue siendo lento

### Lo mas eficiente que se puede hacer con numpy es de la siguiente forma:

In [ ]:
%%time
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

apuestas = np.zeros((combinaciones, 50))
apuestas_num = np.random.rand(combinaciones, 50).argpartition(5,axis=1)[:,:5]
index = np.tile(np.expand_dims(np.arange(combinaciones), axis=0).transpose(), (1, 5))
apuestas[index, apuestas_num] = 1

num_aciertos = apuestas @ combi_ganadora
unique, counts = np.unique(num_aciertos, return_counts=True)

In [ ]:
print(f"{unique} {counts}")

Podemos medirlo varias veces

In [ ]:
%%timeit
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

apuestas = np.zeros((combinaciones, 50))
apuestas_num = np.random.rand(combinaciones, 50).argpartition(5,axis=1)[:,:5]
index = np.tile(np.expand_dims(np.arange(combinaciones), axis=0).transpose(), (1, 5))
apuestas[index, apuestas_num] = 1

num_aciertos = apuestas @ combi_ganadora
unique, counts = np.unique(num_aciertos, return_counts=True)

133ms en mi ordenador aprox